<a href="https://colab.research.google.com/github/m-fila/uczenie-maszynowe-2021-22/blob/main/05_Bayes_irysy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naiwny klasyfikator Bayesa
Autor: Jarosław Żygierewicz

Z klasyfikatorem tym zapoznamy się próbując klasyfikować gatunki irysów. Jest to klasyczny już problem, często wykorzystywany przy porównywaniu różnych technik klasyfikacji. Więcej o pochodzeniu tych danych i problemie można przeczytać tu: [Iris_flower_data](https://en.wikipedia.org/wiki/Iris_flower_data_set)

Kod napiszemy w oparciu o implementacje klasyfikatora Bayesa z biblioteki [scikit-learn](http://scikit-learn.org/stable/about.html#citing-scikit-learn)

Zaczerpniemy stamtąd:
* obiekt klasyfikatora [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
* zbiór danych
* funkcje do oceny jakości 

Przygotopwanie śrdowiska: import wymaganych pakietów.

In [ ]:
import matplotlib
import numpy as np

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

## Zbiór danych irys
Zapoznajemy się z danymi i wybieramy ich podzbiór do dalszej zabawy. Dane to standardowy zestaw dostępny w bibliotece sklearn.

In [ ]:
#Wczytywanie danych
iris = datasets.load_iris() #https://en.wikipedia.org/wiki/Iris_flower_data_set

#Wypisywanie zawartości wczytanego obiektu
print(dir(iris))

Proszę zobaczyć co zawierają dane. Proszę wypisać:

* nazwy gatunków
* kodowanie gatunków, czyli liczby przypisane do nazw gatunków "target"
* nazwy zmiennych oppisujących gatunki "feature_names"
* opis danych "DESCR"

In [ ]:
print(...)
print(...)
print(...)
print(...)

Proszę utworzyć obiekt DataFrame zawierający własności ("features"). 
* proszę przypisać poprawne nazwy kolumn: ```columns=iris.feature_name```
* prodzę dodać do obiektu DataFrame kolumnę etykiet
* proszę wypisać zawartość obiektu DataFrame

In [ ]:
df = pd.DataFrame(...)
df["label"] = ...
print(...)

Proszę wypisać własności przykładu o indeksie 5. 

* proszę zkorzystać z funkcji DataFrame.loc(index)

Wynik powinien być następujący:
```
sepal length (cm)    5.4
sepal width (cm)     3.9
petal length (cm)    1.7
petal width (cm)     0.4
label                0.0
Name: 5, dtype: float64
```

In [ ]:
print(...)

Proszę sprawdzić jaki gatunek odpowiada wartości etykiety przykładu o indekscie 5.
Wynik powinien być następujący:
```
setosa
```

In [ ]:
print(...)

## Analiza wizualna danych. 

Pierwszy krok przy analizie danych z użyciem dowolnego algorytmu to ich inspekcja. Korzystając z metod klasy DataFrame proszę:
* narysować rozkłady wszystkich zmiennych wejściowych

In [ ]:
df....

Naszym zadaniuem jest rozdzielenie gatunków na podstawie wartości własności. 

* proszę narysować rozkłady (histogramy) własności w podziale na  gatunki. Rysując histogramy proszę dobrać zakresy przedziałów na podstawie histogramów narysowanych bez podziału na gatunki
* na podstawie histogramów proszę zitentyfikować cechy, które najlepiej odróżniają gatunki

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10))
color_map = {-1: (1, 1, 1), 0: (0, 0, .9), 1: (1, 0, 0), 2: (.5, .5, 0)}

sns.histplot(data=..., x=..., hue="label", binrange=(4,8),palette=color_map, bins=10, ax=axes[0,0]);
...
...
...

## Wykres skrzypcowy ("violin plot")

Histogramy dla poszczególnych gatunków się nakładają częściowo. Inny sposób wizualizacji to wykres skrzypcowy.

* proszę narysować wykresy skrzypcowe dla poszzcególnych cech, z podziałem na gatunki
* na podstawie rysunków proszę zitentyfikować cechy, które najlepiej odróżniają gatunki

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10))

sns.violinplot(data=..., x="label", y="sepal length (cm)",ax=axes[0,0]); 

## Analiza dwuwymiarowa

Rysunki narysowane do tej pory były jednowymiarowe. Kolejny krok to wizualna analiza korelacji międzu cechami.

* proszę obliczyć macierz kowiariancji między wszystkimi kolumnami danych włączając etykiety
* proszę narysować macierz kowariancji

In [ ]:
corelationMatrix = df....

cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corelationMatrix, square=True, linewidths=.5, cbar_kws={"shrink": .5});

Macierz kowariancji często jest mało czytelna. Wygodniejsze do analizy są rozkłady dwuwymiarowe - na osiach rysunku odkładamy dwie wybrane cechy. 
Proszę uruchomić kod w komórce poniżej i zanalizować korelacje między cechami.
Więcej przykładów wyzializacji można znaleźć [tutaj](https://seaborn.pydata.org/tutorial/axis_grids.html)

In [ ]:
graph = sns.PairGrid(df, hue="label", palette=color_map)
graph.map_diag(sns.histplot)
graph.map_offdiag(sns.scatterplot);
graph.add_legend();

## Przygotowanie danych wejściowych

Aby ułatwić analizę wyniku działania modelu ograniczymy się do dwu cech - wtedy będzie można narysować podział na gatunki na płaszczyźnie.
Znoprmalizujemy też wartości cech: 

\begin{equation}
x_{norm} = \frac{x - \mu_{x}}{\sigma_{x}}
\end{equation}

* proszę utworzyć dane zawierające znormalizowane kolumny: 
* proszę utworzyć dane zawierające tylko wybrane cechy, oraz etykiety: ```["sepal width (cm)","petal length (cm)", "label"]```
* proszę wypisać zawartość nowych danych

**Uwaga:** proszę nie normalizować wartości etykiet

In [ ]:
df_normalized = ...

df_subset = ...
print(df_subset)

Proszę narysować rozkłady dwuwymiarowe dla nowych danych (danych z dwiema cechami).

In [ ]:
graph = ...

## Tworzymy i uczymy klasyfikator

Poniżej znajduje się kod służący klasyfikacji. Proszę go uzupełnić zgodnie z komentarzami i dokumentacją:
[naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) oraz [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

In [ ]:
from sklearn.naive_bayes import GaussianNB
# stwórz instancję klasyfikatora Gaussian Naive Bayes
gnb = GaussianNB()
# wpisz tutaj wartości klas ze zbioru danych
X = ...
Y = ...
# Dopasuj parametry klasyfikatora 
gnb.fit(X, Y);


* proszę wypisać parametry dopasowanie: średnie i odchylenia standardowe rozkładów Gaussa. Średnie tych rozkładów znajdująsię w polu  ```gnb.theta_```, a wariancje
  w ```gnb.sigma_```
* prosze wypisać średnie i wariancje rozkładów cech w podziale na gatunki

**Wskazówka:** proszę skorzystać z grupowania danych: DataFrame.groupby()

Oczekiwany wynik dla parametrów to:
```
Mean:
 [[ 0.85041372 -1.30063009]
 [-0.65922358  0.28437121]
 [-0.19119013  1.01625888]]
Sigma:
 [[0.86093968 0.09738785]
 [0.71270594 0.26351816]
 [0.73246175 0.30949324]]
```

Proszę porównać uzyskane wartości. Czy liczby są zgodne z oczekiwaniami? 

In [ ]:
print("Mean:\n",...)
print("Sigma:\n",...)
print(df_subset....)
print(df_subset....)

#### Porównanie dopasowanych rozkładów z danymi.
Komórki poniżej zawierają funkcje:

* ```plot_2d_gaussian(mean, covariance, axis)``` - funkcja która rysuje dwuwymiarowy rozkłąd Gaussa o średniej ```mean``` i diagonalej macierzy kowariancji ```np.diag(cov)``` na obszarze danymn przez obiekt ```axis```
* ```plot_2d_divison(model, axis)``` - funkcja która rysuje kontury podziału na klasy na podstawie modelu ```model``` na obszarze danymn przez obiekt ```axis```

In [ ]:
def plot_2d_gaussian(mean, cov, axis):
    xx, yy = np.mgrid[-2:2:0.1, -2:2:0.1]
    pos = np.dstack((xx, yy))
    zz = multivariate_normal.pdf(pos, mean=mean, cov=np.diag(cov))
    axis.contour(xx, yy, zz, 10) 
    
def plot_2d_divison(model, axis):
    xx, yy = np.mgrid[-3:3:0.1, -3:3:0.1]  
    pos = np.c_[xx.ravel(), yy.ravel()]
    zz = model.predict(pos)
    zz = np.reshape(zz, xx.shape)
    axis.contour(xx, yy, zz, cmap=plt.cm.Paired)

Proszę narysować:

* dwuwymiarowy rozkład danych dla analizowanego zbioru, korzystając z funkcji ```sns.jointplot```. Punkty powinny być pokolorowane według przyneleżności do gatunku.
* korzystając z funkcji ```plot_2d_gaussian(mean, covariance, axis)``` na tym samym wykresie nanieść rozkłady Gaussa dopasowane przez model ```gnb = GaussianNB()```

In [ ]:
graph = sns.jointplot(...)

classNumber = 0
plot_2d_gaussian(..., axis=graph.ax_joint)  

Proszę narysować:

* dwuwymiarowy rozkład danych dla analizowanego zbioru, korzystając z funkcji ```sns.jointplot```. Punkty powinny być pokolorowane według przyneleżności do gatunku.
* korzystając z funkcji ```plot_2d_divison(model, axis)``` na tym samym wykresie nanieść granice podziału na gatunki

In [ ]:
graph = sns.jointplot(...)

plot_2d_divison(..., axis=graph.ax_joint)

## Analiza jakości modelu
Korzystająć z walidacji krzyżowej przeprowadzimy analizę miar jakości modeli. Skorzystamy z funkcji dostrczanych przez [Model evaluation](http://scikit-learn.org/stable/modules/model_evaluation.html)
* upewnij się, że dokładnie rozumiesz co zwracają te funkcje [(sprawdź w dokumentacji)]
* porównaj z definicjami z [wykładu](https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wyk%C5%82ad_Ocena_jako%C5%9Bci_klasyfikacji) 
* proszę uzupełnić kod funkcji ```printScores(model, X, Y)``` - która przyjmuje dopasowany ```model```, cechy ```X``` oraz etykiety ```Y```
* korzystając z funkcji ```printScores(model, X, Y)``` proszę wypisać wartości miar jakości dla modelu dopasowanego w poprzednich komórkach.

In [ ]:
def printScores(model, X, Y):
    # użyj classification_report() żeby policzyć najpopularniejsze miary 
    print("Classification report:")
    print(classification_report(Y, model.predict(X)))
    # wypisz macierz pomyłek 
    print("Confusion matrix:")
    print(confusion_matrix(...)
    
printScores(...)  

## Porównanie  modeli


Korzystając z kodu napisanego w poprzednich komórkach stwórz trzy modele klasyfikatorów:
* pierwszym niech będzie korzystał z cech ```["sepal width (cm)","petal length (cm)"]``` (nasz dotychczasowy model)
* drugim niech korzysta tylko z cech ```["sepal length (cm)","sepal width (cm)"]```
* trzeci niech korzysta ze wszytkich czterech cech
* dla każdego z modeli wypisz wartości miar jakości. Prównaje te wartości między modelami.

In [ ]:
df_subset = ...
...
...
gnb_features_1_2 = GaussianNB()

...

In [ ]:
df_subset = ...
...
...
gnb_features_0_1 = GaussianNB()
...
...

In [ ]:
df_subset = ...
...
...
gnb_features_all = GaussianNB()
...
...

## Analiza wpływu normalizacji danych.

* proszę wytenować model na wszytkich cechach, ale korzystając z oryginalnych danych - bez normalizacji wartości cech
* dla tego modelu proszę wypisać wartości miar jakości

Czy normalizacja wpływa na wydajność naszego modelu?

In [ ]:
# Stwórz model 4, bez normalizacji danych
df_subset = ...
...
...
# dofituj parametry klasyfikatora 
gnb_features_all = GaussianNB()
...
...